In [3]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
# Load data fetched from SEC API using sec_edgar_fetcher.py
df_edgar = pd.read_csv("../Data/sec_data/pivoted_financial_data_2010_full.csv")

In [34]:
# Extract the missing values, will be used to help with feature selection
missing_edgar = df_edgar.isna().sum()
missing_edgar = missing_edgar.sort_values()
missing_edgar.head(30)

ticker                    0
date                      0
IncomeTax               900
DilutedShares          1914
WeightedShares         1974
Assets                 2380
Liabilities            2445
NetIncome              2800
FinancingCashFlow      3173
InvestingCashFlow      3199
OperatingCashFlow      3304
StockholdersEquity     4072
PPE                    5262
Goodwill               6408
CurrentAssets          6838
CurrentLiabilities     6850
OperatingIncome        7182
ShareBasedComp         7464
InterestExpense        8732
CapEx                  8748
Revenue               10527
SGA                   12655
Amortization          14025
GrossProfit           16408
COGS                  16964
LongTermDebt          17783
OperatingExpenses     21111
RnD                   21294
dtype: int64

In [35]:
df_edgar.columns

Index(['ticker', 'date', 'Amortization', 'Assets', 'COGS', 'CapEx',
       'CurrentAssets', 'CurrentLiabilities', 'DilutedShares',
       'FinancingCashFlow', 'Goodwill', 'GrossProfit', 'IncomeTax',
       'InterestExpense', 'InvestingCashFlow', 'Liabilities', 'LongTermDebt',
       'NetIncome', 'OperatingCashFlow', 'OperatingExpenses',
       'OperatingIncome', 'PPE', 'Revenue', 'RnD', 'SGA', 'ShareBasedComp',
       'StockholdersEquity', 'WeightedShares'],
      dtype='object')

In [36]:
# Transforming json cik file to csv for easier use
import json

json_data = pd.read_json("../Data/company_tickers.json").T
json_data.columns = ["cik", "ticker", "company_name"]
json_data.to_csv("../Data/company_tickers.csv", index=False)

In [42]:
# Reading data on 345 filings (insider trades) from Q1 2010, 2019, and 2024.
# This data contains exact tickers which allows me to get CIKs for companies that are missing in the edgar data.

df345_2010 = pd.read_csv("../Data/345-filings/SUBMISSION_2010Q1.tsv", sep='\t')
df345_2019 = pd.read_csv("../Data/345-filings/SUBMISSION_2019Q1.tsv", sep='\t')
df345_2024 = pd.read_csv("../Data/345-filings/SUBMISSION_2024Q1.tsv", sep='\t')

# Select relevant columns and drop duplicates
df345_2010 = df345_2010[['ISSUERCIK', 'ISSUERNAME', 'ISSUERTRADINGSYMBOL']].drop_duplicates()
df345_2019 = df345_2019[['ISSUERCIK', 'ISSUERNAME', 'ISSUERTRADINGSYMBOL']].drop_duplicates()
df345_2024 = df345_2024[['ISSUERCIK', 'ISSUERNAME', 'ISSUERTRADINGSYMBOL']].drop_duplicates()

# Rename columns for consistency
df345_2010.columns = ["cik", "company_name", "ticker"]
df345_2019.columns = ["cik", "company_name", "ticker"]
df345_2024.columns = ["cik", "company_name", "ticker"]

# Combine dataframes
df_combined = pd.concat([df345_2010, df345_2019, df345_2024])

# Drop duplicates to get unique rows
df_unique_ciks = df_combined.drop_duplicates()

df_unique_ciks

,cik,company_name,ticker
0,355627,CTI GROUP HOLDINGS INC,CTIG
1,1289592,"Tower Group, Inc.",TWGP
2,1022345,MARINER ENERGY INC,ME
3,1401688,"Vitacost.com, Inc.",VITC
4,857005,PARAMETRIC TECHNOLOGY CORP,PMTC
...,...,...,...
67107,1064722,GIVEMEPOWER CORP,GMPW
67126,1832487,"Guerrilla RF, Inc.",GUER
67134,1809750,Edible Garden AG Inc,EDBL
67239,1834032,Constellation Acquisition Corp I,CSTAF


In [43]:
# Create final file with all company tickers and ciks combined.
company_tickers = pd.read_csv("../Data/company_tickers.csv")
combined_df = pd.concat([df_unique_ciks, company_tickers]).drop_duplicates().reset_index(drop=True)
combined_df.to_csv("../Data/combined_company_tickers.csv", index=False)
combined_df

,cik,company_name,ticker
0,355627,CTI GROUP HOLDINGS INC,CTIG
1,1289592,"Tower Group, Inc.",TWGP
2,1022345,MARINER ENERGY INC,ME
3,1401688,"Vitacost.com, Inc.",VITC
4,857005,PARAMETRIC TECHNOLOGY CORP,PMTC
...,...,...,...
15122,2032528,Roman DBDR Acquisition Corp. II,DRDBW
15123,2028027,Newbury Street II Acquisition Corp,NTWOW
15124,2028027,Newbury Street II Acquisition Corp,NTWOU
15125,2023554,Sandisk Corp,SNDKV


In [7]:
# Tag analysis
tags = pd.read_csv("../Data/sec_data/tag_frequencies.csv")
sp500_2010 = pd.read_csv("../Data/sp500_tickers_2010.csv")

# Basic coverage analysis
total_companies = sp500_2010.shape[0]
tags['coverage_percentage'] = (tags['company_count'] / total_companies * 100).round(2)

# Show top 20 tags by coverage
top_coverage = tags.sort_values('coverage_percentage', ascending=False)
print("\nTop 20 tags by coverage:")
print(top_coverage[['tag', 'coverage_percentage', 'company_count', 'avg_values_per_company']].head(20))


Top 20 tags by coverage:
                                                tag  coverage_percentage  \
0                                            Assets                97.60   
1                  LiabilitiesAndStockholdersEquity                97.60   
2        NetCashProvidedByUsedInFinancingActivities                97.39   
3        NetCashProvidedByUsedInInvestingActivities                97.39   
4                           IncomeTaxExpenseBenefit                97.39   
5        NetCashProvidedByUsedInOperatingActivities                96.59   
8     WeightedAverageNumberOfSharesOutstandingBasic                96.39   
7                             EarningsPerShareBasic                96.39   
6   WeightedAverageNumberOfDilutedSharesOutstanding                96.39   
10                          EarningsPerShareDiluted                96.19   
9                RetainedEarningsAccumulatedDeficit                96.19   
11                                    NetIncomeLoss           

In [9]:
total_companies = sp500_2010.shape[0]
total_companies

499